In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline

In [2]:
data_path = "/home/akash/projects/AML/ds_comptetions/kaggle_sales_predict/data/"
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
print onlyfiles

['shops.csv', 'test.csv', 'submission.csv', 'sales_train.csv', 'item_categories.csv', 'sample_submission.csv', 'submission_1.167.csv', 'items.csv']


In [3]:
categories = pd.read_csv(data_path+"item_categories.csv")

In [4]:
categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [5]:
categories.shape

(84, 2)

In [6]:
items = pd.read_csv(data_path+"items.csv")
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [7]:
items.shape

(22170, 3)

In [8]:
shops = pd.read_csv(data_path+"shops.csv")

In [9]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [10]:
shops.shape

(60, 2)

In [11]:
train = pd.read_csv(data_path+"sales_train.csv")
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [13]:
train.shape

(2935849, 6)

In [14]:
test = pd.read_csv(data_path+"test.csv")

In [15]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [16]:
test.shape

(214200, 3)

In [17]:
sample_submit = pd.read_csv(data_path+"sample_submission.csv")
sample_submit.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


### Create dumb benchmarks

In [18]:
test_all_zeroes = test[["ID"]] # if we take only a single ssuare brackets we get a series not a DF
test_all_zeroes["item_cnt_month"] = 0
test_all_zeroes.to_csv("all_0.csv",index=False)

/home/akash/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
test_all_zeroes.head()

,ID,item_cnt_month
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


### all_0 benchmark = 1.25

In [20]:
test_all_ones = test[["ID"]]
test_all_ones["item_cnt_month"] = 1
test_all_ones.to_csv("all_1.csv",index=False)

/home/akash/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### all_1s becnhmark = 1.41214

* score_last_month = 1.167
* all_0s = 1.25
* all_1s = 1.41214

### Define Transformers

In [21]:
from functools import wraps
def transformer_decorator(fun):
    @wraps(fun)
    def wrapped(df,**kwargs):
        print("\n\n Applying transformer: %s"%fun.__name__)   
        df = fun(df,**kwargs)
        print df.head()
        return df
    return wrapped

In [22]:
@transformer_decorator
def transform_date(df):
    df["date"] = pd.to_datetime(df["date"], format="%d.%m.%Y")
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df["day"] = df["date"].dt.day
    return df

date_transformer = FunctionTransformer(transform_date, validate=False)

In [23]:
@transformer_decorator
def calculate_daily_sales(df):
    df["daily_sale"] = df["item_price"] * df["item_cnt_day"]
    return df

daily_sale_transformer = FunctionTransformer(calculate_daily_sales, validate=False)

In [24]:
@transformer_decorator
def aggregate_monthly_sales(df):
    df = train.groupby(["shop_id", "item_id", "date_block_num"], as_index=False).item_cnt_day.sum().\
    rename(columns={"item_cnt_day":"item_cnt_month"})
    return df
monthly_aggregate_transformer = FunctionTransformer(aggregate_monthly_sales, validate=False)

In [25]:
@transformer_decorator
def clip_monthly_sales(df):
    total = len(df)
    count_20 = df.loc[(df["item_cnt_month"]>20)].shape[0]
    count_0 = df.loc[(df["item_cnt_month"]<0)].shape[0]
    percent_20 = count_20/float(total)
    percent_0 = count_0/float(total)
    print "%f percent is less than 0"%percent_0
    print "%f percent is greater than 20"%percent_20
    df["item_cnt_month"] = df["item_cnt_month"].clip(0,20)
    return df

clip_sales_transformer = FunctionTransformer(clip_monthly_sales, validate=False)

In [26]:
# @transformer_decorator
# def add_last_month_sales(df):
#     df_last = df.loc[train["date_block_num"]==33]
#     last_month_dict = 

### Build Processing Pipeline

In [27]:
preprocess_pipeline = make_pipeline(
                        date_transformer, 
                        daily_sale_transformer, 
                        monthly_aggregate_transformer,
                        )

In [28]:
train = preprocess_pipeline.transform(train)



 Applying transformer: transform_date
        date  date_block_num  shop_id  item_id  item_price  item_cnt_day  \
0 2013-01-02               0       59    22154      999.00           1.0   
1 2013-01-03               0       25     2552      899.00           1.0   
2 2013-01-05               0       25     2552      899.00          -1.0   
3 2013-01-06               0       25     2554     1709.05           1.0   
4 2013-01-15               0       25     2555     1099.00           1.0   

   year  month  day  
0  2013      1    2  
1  2013      1    3  
2  2013      1    5  
3  2013      1    6  
4  2013      1   15  


 Applying transformer: calculate_daily_sales
        date  date_block_num  shop_id  item_id  item_price  item_cnt_day  \
0 2013-01-02               0       59    22154      999.00           1.0   
1 2013-01-03               0       25     2552      899.00           1.0   
2 2013-01-05               0       25     2552      899.00          -1.0   
3 2013-01-06        

### Make Last Month Transformer

In [29]:
from sklearn.base import BaseEstimator, RegressorMixin

class LastMonthSaleTransformer(BaseEstimator, RegressorMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        X_last = X.loc[train["date_block_num"]==33]
        last_month_dict = X_last.groupby(["shop_id", "item_id"])["item_cnt_month"].apply(list).to_dict()
        self.last_month_dict_ = last_month_dict
        return self
    
    def transform(self, X, y=None):
        last_month_dict = self.last_month_dict_
        
        def fill_last_month(key):
            if key in last_month_dict:
                return last_month_dict[key][0]
            else:
                return np.NaN
        
        X["last_month_sales"] = X.apply( lambda row: fill_last_month((row["shop_id"], row["item_id"])) , axis=1)
        X["last_month_sales"] = X["last_month_sales"].fillna(0)
        return X

In [30]:
last_month_transformer = LastMonthSaleTransformer()
last_month_transformer.fit(train)
train = last_month_transformer.transform(train)

In [31]:
train

,shop_id,item_id,date_block_num,item_cnt_month,last_month_sales
0,0,30,1,31.0,0.0
1,0,31,1,11.0,0.0
2,0,32,0,6.0,0.0
3,0,32,1,10.0,0.0
4,0,33,0,3.0,0.0
5,0,33,1,3.0,0.0
6,0,35,0,1.0,0.0
7,0,35,1,14.0,0.0
8,0,36,1,1.0,0.0
9,0,40,1,1.0,0.0


In [32]:
train.loc[train["date_block_num"] == 33]

,shop_id,item_id,date_block_num,item_cnt_month,last_month_sales
8113,2,31,33,1.0,1.0
8205,2,486,33,3.0,3.0
8254,2,787,33,1.0,1.0
8284,2,794,33,1.0,1.0
8367,2,968,33,1.0,1.0
8378,2,988,33,1.0,1.0
8422,2,1075,33,1.0,1.0
8436,2,1121,33,1.0,1.0
8467,2,1377,33,1.0,1.0
8476,2,1387,33,1.0,1.0


In [33]:
last_month_transformer.last_month_dict_[(2,486)]

[3.0]

In [34]:
test = last_month_transformer.transform(test)

In [35]:
test["item_cnt_month"] = test["last_month_sales"]
test["item_cnt_month"] = test["item_cnt_month"].clip(0,20)

In [36]:
test.head(50)

,ID,shop_id,item_id,last_month_sales,item_cnt_month
0,0,5,5037,0.0,0.0
1,1,5,5320,0.0,0.0
2,2,5,5233,1.0,1.0
3,3,5,5232,0.0,0.0
4,4,5,5268,0.0,0.0
5,5,5,5039,1.0,1.0
6,6,5,5041,2.0,2.0
7,7,5,5046,0.0,0.0
8,8,5,5319,0.0,0.0
9,9,5,5003,0.0,0.0


### Create last month sale benchmark

In [37]:
test[["ID", "item_cnt_month"]].to_csv("last_month.csv", index=False)

### Score =1.167